In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
def get_homepage(i):
    url="https://etfdb.com/etf/"+all_ETF[i]
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    span=soup.find_all("a",text='Home page')
    url_homepage=span[0].get("href")
    return url_homepage
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - dt.timedelta(1)
    year = start_date.year
    month = start_date.month
    while (temp < end_date):
        temp = temp + dt.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    for i in range(len(ans)):
        if ans[i].isoweekday()>5:
            ans[i]=ans[i]+dt.timedelta(days=5-ans[i].isoweekday())
    return ans
last_day_of_month = get_the_last_day_of_month(dt.datetime(2016,1,1,0,0),dt.datetime(2018,12,31,0,0))
#dealing with 3/30 have no NAV
last_day_of_month[-10]=last_day_of_month[-10]+dt.timedelta(days=-1)

In [2]:
need_list=pd.read_excel("new_list.xlsx")
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values
all_ETF

array(['EWM', 'ADRE', 'EEM', 'FXI', 'PGJ', 'VWO', 'EEV', 'EUM', 'SPEM',
       'GXC', 'GMF', 'FXP', 'DEM', 'DGS', 'FNI', 'AIA', 'PIE', 'HAO',
       'EPI', 'THD', 'PIN', 'AAXJ', 'EWX', 'EDC', 'EDZ', 'IDX', 'EET',
       'XPP', 'YINN', 'YANG', 'CQQQ', 'CHIC', 'CHIX', 'YAO', 'VNM', 'INDY',
       'CHII', 'CHIQ', 'CHIE', 'YXI', 'EIDO', 'SCIN', 'SCIF', 'ECNS',
       'EPHE', 'PEK', 'INDL', 'INXX', 'CHIM', 'ASEA', 'EDIV', 'MCHI',
       'FEM', 'FCA', 'DBEM', 'INCO', 'EEMV', 'INDA', 'NFTY', 'FEMS',
       'SMIN', 'EEMA', 'CXSE', 'DGRE', 'ASHR', 'KFYP', 'KWEB', 'AXJV',
       'KBA', 'CN', 'ASHS', 'CNXT', 'EMQQ', 'XCEM', 'CHAU', 'CHAD', 'EMGF',
       'ASHX', 'CNHX', 'XINA', 'KALL', 'AFTY'], dtype=object)

In [3]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [20]:
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()
def get_single_symbol(symbol,browser,wait = 3):
    
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')
    
    filepath = f'./data_yahoo/{symbol}.csv'
    if os.path.isfile(filepath):
        print('File exist')
        return 
    
    url = f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}'
    browser.get(url)
    browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
    start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
    start_date.clear()
    start_date.send_keys('1/1/2016')
    end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
    end_date.clear()
    end_date.send_keys('1/1/2019')

    try:
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
    except:
        browser.find_element_by_css_selector("[class=' Bd Bdc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) D(ib) C($c-fuji-blue-1-b) Bdc($actionBlueHover):h C($actionBlueHover):h Cur(p) Td(n)  Py(8px) cancel Miw(80px)! Fl(end)']").click()
        browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()

    browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
    time.sleep(5)
    browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
    time.sleep(wait)

    while not os.path.isfile(filepath):
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
        time.sleep(wait)

    print('Download',symbol)
def get_all_symbol_csv(dic):
    
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')

    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        get_single_symbol(s,browser)

    browser.quit()
def combine_data(dic,last_day_of_month):
    com_data = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
        
    for s in dic['symbol']:
        data = pd.read_csv(f'./data_yahoo/{s}.csv')
        data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
        
        # Mask
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        
        data_close = data[['Date','Close']].reset_index(drop=True)
        data_close = data_close.rename(columns={'Close':s})
        com_data = com_data.merge(data_close,on='Date',how='left')
        
    # check directory
    if not os.path.isdir('./Combine_data'):
        os.mkdir('./Combine_data')
    com_data.to_csv('./Combine_data/com_data.csv')
    
    return com_data
def combine_daily_data(dic):
    com_data=pd.DataFrame()
    for s in dic['symbol']:
        data = pd.read_csv(f'./data_yahoo/{s}.csv',index_col=0,parse_dates=True)["Adj Close"]
        com_data=pd.concat([com_data,data],axis=1)
    com_data.columns=dic['symbol']
    com_data=com_data.loc["20160101":"20190101"]
    return com_data
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()

In [5]:
#get yahoo_finance_data
get_all_symbol_csv(dic)

File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist


In [6]:
"""
#combine_all_data
com_data = combine_data(dic,last_day_of_month)
com_data.index=com_data["Date"]
com_data=com_data.iloc[::-1,1:]
"""

In [26]:
com_data=combine_daily_data(dic)

In [28]:
daily_date=com_data.index

In [2]:
#將太晚發行ETF拿掉
"""
origin_list=pd.read_excel("origin_list.xlsm",index_col=22,parse_dates=True)
need_list=origin_list.sort_index().loc[:"2016-01-01"]
need_list.to_excel("new_list.xlsx")
"""

In [133]:
"""
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))
dic = {'symbol':[],'name':[],'addr':[]}
for i,s in enumerate(symbol_list):
    dic['symbol'].append(s)
    dic['name'].append(name_list[i])
    dic['addr'].append(homepage_list[i])
    print(s,name_list[i],homepage_list[i])
    
 # Save this dic
with open("ETF_dic.pkl","wb") as f:
    pickle.dump(dic,f)
"""

'\nhomepage_list = []\nfor i in symbol_list:\n    url = \'https://www.etf.com/\'+i+\'#overview\'\n    res = requests.get(url)\n    doc = pq(res.text)\n    doc.make_links_absolute(res.url)\n    homepage_list.append(doc(\'.field-content.helplink\')(\'a\').attr(\'href\'))\ndic = {\'symbol\':[],\'name\':[],\'addr\':[]}\nfor i,s in enumerate(symbol_list):\n    dic[\'symbol\'].append(s)\n    dic[\'name\'].append(name_list[i])\n    dic[\'addr\'].append(homepage_list[i])\n    print(s,name_list[i],homepage_list[i])\n    \n # Save this dic\nwith open("ETF_dic.pkl","wb") as f:\n    pickle.dump(dic,f)\n'

In [44]:
#print all ETF address
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        print (url_homepage)
    except:
        print(all_ETF[i])     

http://us.ishares.com/product_info/fund/overview/EWM.htm?qt=EWM
https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=ADRE
http://us.ishares.com/product_info/fund/overview/EEM.htm?qt=EEM
http://us.ishares.com/product_info/fund/overview/FXI.htm?qt=FXI
http://invescopowershares.com/products/overview.aspx?ticker=PGJ
https://personal.vanguard.com/us/funds/snapshot?FundId=0964&FundIntExt=INT
http://www.proshares.com/funds/eev.html
http://www.proshares.com/funds/eum.html
https://us.spdrs.com/en/etf/spdr-portfolio-emerging-markets-etf-SPEM
https://www.spdrs.com/product/fund.seam?ticker=GXC
https://www.spdrs.com/product/fund.seam?ticker=GMF
http://www.proshares.com/funds/fxp.html
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=50
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=53
http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://us.ishares.com/product_info/fund/overview/AIA.htm?qt=AIA
http://invescopowershares.com/product

In [33]:
# print unsolve website
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        if "ishares" in url_homepage:#處理ishare的ETF
            continue
        elif "invesco" in url_homepage:#處理invesco的ETF
            continue
        elif "vanguard" in url_homepage: #vanguard
            continue
        elif "proshares" in url_homepage: #proshares
            continue
        elif "spdrs" in url_homepage: #spdrs
            continue
        elif "wisdomtree" in url_homepage: #wisdomtree
            continue
        elif "vaneck" in url_homepage:#vaneck
            continue
        elif "emerging" in url_homepage:#emerging
            continue
        elif "globalxfunds" in url_homepage:#globalxfunds
            continue
        else:
            print (url_homepage)
    except:
        print(all_ETF[i])
        

http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://www.guggenheimfunds.com/etf/fund/HAO
http://direxionshares.com/etf/embu_bull_3x_shares.html
http://direxionshares.com/etf/embe_bear_3x_shares.html
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://guggenheiminvestments.com/products/etf/cqqq
CHIC
http://guggenheiminvestments.com/products/etf/yao
http://www.direxionshares.com/etf/indiabu_2x_shares.html
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FEM
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FCA
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/DBEM
NFTY
http://www.ftportfolios.com/retail/etf/etfsummary.aspx?Ticker=FEMS
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHR
http://kraneshares.com/kfyp/
http://kraneshares.com/kweb/
http://kraneshares.com/kba/
https://etfus.deutscheawm.com/US/EN/Product-Detail-

In [29]:
#ishare 未能讀檔 0
def download_ishare(i,url_homepage):
    resp = requests.get(url_homepage)
    soup = BeautifulSoup(resp.text, 'html.parser')
    excel_file=soup.find_all("a",{"class":"icon-xls-export"})[3].get("href")
    domain_url = "https://www.ishares.com"
    excel_file= domain_url+excel_file
    file_name="data/"+all_ETF[i]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)
#invesco 1
def download_invesco(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Historical NAVs").click()
    time.sleep(5)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".csv"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    NAV=pd.read_csv("data/"+all_ETF[i]+".csv",index_col=2,parse_dates=True).loc[daily_date]["NAV"]
    return NAV
#vanguard 5
def get_table(search_url,beginDate,endDate):
    driver = webdriver.Chrome()
    driver.get(search_url)
    driver.find_element_by_name("beginDate").send_keys(beginDate)
    driver.find_element_by_name("endDate").send_keys(endDate)
    driver.find_element_by_id("_cbdExt8").click()
    page_content=pd.read_html(driver.current_url)
    data_table=page_content[1].iloc[54:,:2]
    data_table=pd.DataFrame(data_table[1].values,index=pd.to_datetime(data_table[0]),columns=["Nav"])
    driver.quit()
    return  data_table
def download_vanguard(i,url_homepage):
    url_homepage=get_homepage(i)
    search_url="https://personal.vanguard.com/us/funds/tools/pricehistorysearch?"+url_homepage[url_homepage.find("FundId"):url_homepage.find("FundIntExt")-1]
    data_table_2016=get_table(search_url,"01/01/2016","12/31/2016")
    data_table_2017=get_table(search_url,"01/01/2017","12/31/2017")
    data_table_2018=get_table(search_url,"01/01/2018","12/31/2018")
    NAV=pd.concat([data_table_2016,data_table_2017,data_table_2018])
    NAV=NAV.loc[daily_date]
    return NAV["Nav"]
#proshares 6
def download_proshares(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("NAV History").click()
    time.sleep(10)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    filename=all_ETF[i]+'.csv'
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,filename))
    NAV=pd.read_csv("data/"+all_ETF[i]+'.csv',index_col=0,parse_dates=True).loc[daily_date]["NAV"]
    return NAV
#spdrs 9
def download_spdrs(i,url_homepage):
    domain_url = "https://us.spdrs.com/"
    excel_file= domain_url+"site-content/xls/"+all_ETF[9]+"_HistoricalNav.xls?fund="+all_ETF[9]+"&amp;docname=Most+Recent+NAV+%2F+NAV+History&amp;onyx_code1=&amp;onyx_code2="
    excel_file
    file_name="data/"+all_ETF[i]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)
    file=pd.read_excel(file_name,index_col=0,header=2,parse_dates=True).dropna(axis=1,how="all")[:-12]
    file.index=pd.to_datetime(file.index)
    Nav=file["Nav"].loc[daily_date]
    return(Nav)
#wisdomtree 13
def download_wisdomtree(i,url_homepage):
    url="https://etfdb.com/etf/"+all_ETF[i]
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    span=soup.find_all("a",text='Home page')
    url_homepage=span[0].get("href")
    resp = requests.get(url_homepage)
    soup = BeautifulSoup(resp.text, 'html.parser')
    link_address=soup.find_all("a",{"class":"text-link fund-modal-trigger"})[0].get("data-href")
    data=pd.read_html(link_address,index_col=0,parse_dates=True)
    NAV=data[0].loc[daily_date]["Nav"]
    return NAV
#globalxfunds 32
def download_globalxfunds(i,url_homepage):
    ETF_len=len(all_ETF[i])
    download_link=url_homepage[:-ETF_len]+"fund/"+url_homepage[-ETF_len:]+"/?download_chart_data=true"
    file_name="data/"+all_ETF[i]+".csv"
    urllib.request.urlretrieve(download_link, file_name)
    NAV=pd.read_csv(file_name,index_col=0,header=1,parse_dates=True)["Closing Price"].loc[daily_date]
    return NAV
#emerging 41
def download_emerging(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Download NAV/Premium Discount").click()
    time.sleep(40)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".xls"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    path="data/"+file_name
    day_NAV=pd.read_excel(path,index_col=0,header=4,parse_dates=True).loc[daily_date]
    NAV=day_NAV[day_NAV["Fund Symbol"]==all_ETF[i]]["NAV"]
    return NAV

In [30]:
for i in range(len(all_ETF)):
    try:
        url_homepage=get_homepage(i)
        print (url_homepage)
#        if "ishares" in url_homepage:#處理ishare的ETF
#            download_ishare(i,url_homepage)
        if "invesco" in url_homepage:#處理invesco的ETF
            NAV=download_invesco(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        elif "vanguard" in url_homepage: #vanguard
            NAV=download_vanguard(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        elif "proshares" in url_homepage: #proshares
            NAV=download_proshares(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        elif "spdrs" in url_homepage: #spdrs
            NAV=download_spdrs(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        elif "wisdomtree" in url_homepage: #wisdomtree
            NAV=download_wisdomtree(13,url_homepage)
            com_data[all_ETF[i]]=NAV
 #       if "vaneck" in url_homepage:#vaneck
        elif "emerging" in url_homepage:#emerging
            NAV=download_emerging(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        elif "globalxfunds" in url_homepage:#globalxfunds
            NAV=download_globalxfunds(i,url_homepage)
            com_data[all_ETF[i]]=NAV
    except:
        print(all_ETF[i])
com_data.to_csv("last_data.csv")

http://us.ishares.com/product_info/fund/overview/EWM.htm?qt=EWM
https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=ADRE


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


http://us.ishares.com/product_info/fund/overview/EEM.htm?qt=EEM
http://us.ishares.com/product_info/fund/overview/FXI.htm?qt=FXI
http://invescopowershares.com/products/overview.aspx?ticker=PGJ


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


https://personal.vanguard.com/us/funds/snapshot?FundId=0964&FundIntExt=INT
http://www.proshares.com/funds/eev.html


...

INCO
http://us.ishares.com/product_info/fund/overview/EEMV.htm?fundSearch=true&qt=EEMV
http://us.ishares.com/product_info/fund/overview/INDA.htm?fundSearch=true&qt=INDA
NFTY
http://www.ftportfolios.com/retail/etf/etfsummary.aspx?Ticker=FEMS
http://www.ishares.com/us/products/239660/ishares-msci-india-smallcap-etf?fundSearch=true&qt=SMIN
https://www.ishares.com/us/products/239629/EEMA?referrer=tickerSearch
http://www.wisdomtree.com/etfs/fund-details.aspx?etfid=80
http://www.wisdomtree.com/etfs/fund-details.aspx?etfid=86
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHR
http://kraneshares.com/kfyp/
http://kraneshares.com/kweb/
http://www.ishares.com/us/products/264606/AXJV
http://kraneshares.com/kba/
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/CN
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHS
http://www.vaneck.com/funds/CNXT.aspx
http://www.emqqetf.com/
http://www.emergingglobaladvisors.com/xcem.cfm


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:102: DeprecationWarning: use options instead of chrome_options


XCEM
http://www.direxioninvestments.com/products/direxion-daily-csi-300-china-a-share-bull-2x-shares
http://www.direxioninvestments.com/products/direxion-daily-csi-300-china-a-share-bear-1x-shares
https://www.ishares.com/us/products/272820/ishares-msci-emerging-multi-factor-etf


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:102: DeprecationWarning: use options instead of chrome_options


EMGF
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHX
http://www.csopasset.us/en-us/products/etf/csop-msci-china-a-international-hedged-etf/
https://www.spdrs.com/product/fund.seam?ticker=XINA
KALL
AFTY


In [32]:
com_data.dropna(axis=1).to_excel("Com")

,EWM,ADRE,EEM,FXI,PGJ,VWO,EUM,SPEM,GXC,GMF,...,CNXT,EMQQ,XCEM,CHAU,CHAD,EMGF,ASHX,CNHX,XINA,AFTY
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,26.030420,28.20,29.478195,31.667364,31.37,$31.88,30.26,71.087303,71.087303,71.087303,...,36.221668,22.332735,16.232405,19.092026,45.468994,31.337992,18.760775,29.900549,71.087303,12.157043
2016-01-05,26.238945,28.32,29.544098,31.491177,31.55,$31.90,30.19,70.936196,70.936196,70.936196,...,36.561028,22.594316,16.232405,19.667866,44.496712,31.337992,19.255548,27.508121,70.936196,12.444071
2016-01-06,25.648132,27.78,28.979204,30.804977,31.25,$31.48,30.78,70.411446,70.411446,70.411446,...,36.351421,22.342567,16.232405,19.370018,45.766632,30.187374,19.466434,27.914816,70.411446,12.351482
2016-01-07,25.543871,26.79,28.084784,29.627300,29.82,$30.45,31.71,67.359030,67.359030,67.359030,...,33.317146,21.458504,16.232405,17.036879,47.135761,30.187374,18.120007,27.914816,67.359030,11.684834
2016-01-08,25.300598,26.49,27.783508,29.321291,29.75,$30.38,32.05,67.557209,67.557209,67.557209,...,33.127502,21.339514,16.232405,17.165947,47.016708,30.187374,18.168673,26.444611,67.557209,11.805202
2016-01-11,25.265842,26.40,27.774094,29.163649,28.95,$29.93,32.07,65.357357,65.357357,65.357357,...,31.800009,20.651142,16.232405,16.401470,48.108040,30.187374,17.706345,25.270355,65.357357,11.638539
2016-01-12,25.578629,26.65,27.830582,29.117285,29.50,$30.05,32.00,65.267665,65.267665,65.267665,...,32.528633,21.133001,15.414730,17.066664,47.135761,30.187374,17.955353,25.270355,65.267665,11.721871
2016-01-13,25.300598,26.22,27.538717,28.579447,28.63,$29.76,32.33,65.231799,65.231799,65.231799,...,30.532400,20.778982,15.414730,15.438431,49.437485,30.187374,17.010420,25.270355,65.231799,11.397807
2016-01-14,25.648132,27.00,27.896482,28.987461,29.46,$30.10,31.90,65.193586,65.193586,65.193586,...,32.368935,21.103498,15.441051,16.748959,47.403629,30.187374,17.438681,25.270355,65.193586,11.786683


In [22]:
#vaneck
url="https://etfdb.com/etf/"+all_ETF[25]
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
span=soup.find_all("a",text='Home page')
url_homepage=span[0].get("href")
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1'}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
url_link="https://www.vaneck.com/etf/equity/idx/documents/"
driver.get(url_homepage)
try:
    time.sleep(1)
    driver.find_element_by_link_text("Individual Investor").click()
    driver.find_element_by_link_text("I agree").click()
except:
    time.sleep(1)
time.sleep(3)
driver.find_element_by_link_text("Documents").click()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Documents"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)
